In [1]:
from neo4j import GraphDatabase, exceptions
import pandas as pd
from decouple import config

In [2]:
class ConsultaDadosNeo4j:
    def __init__(self):
        # Carrega as informações do arquivo .env
        self.uri = config('NEO4J_URI')
        self.user = config('NEO4J_USERNAME')
        self.password = config('NEO4J_PASSWORD')
        self._driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
    
    def close(self):
        self._driver.close()

    def consultar_beneficiarios(self):
        with self._driver.session() as session:
            query = "MATCH (b:Beneficiario) RETURN b"
            return session.run(query).data()

    def consultar_ies(self):
        with self._driver.session() as session:
            query = "MATCH (ies:IES) RETURN ies"
            return session.run(query).data()

    def consultar_cursos(self):
        with self._driver.session() as session:
            query = "MATCH (c:Curso) RETURN c"
            return session.run(query).data()

    def consultar_bolsas(self):
        with self._driver.session() as session:
            query = "MATCH (bo:Bolsa) RETURN bo"
            return session.run(query).data()

    def consultar_regioes(self):
        with self._driver.session() as session:
            query = "MATCH (r:Regiao) RETURN r"
            return session.run(query).data()

    def consultar_ufs(self):
        with self._driver.session() as session:
            query = "MATCH (uf:UF) RETURN uf"
            return session.run(query).data()

    def consultar_municipios(self):
        with self._driver.session() as session:
            query = "MATCH (m:Municipio) RETURN m"
            return session.run(query).data()

    def consultar_todos_os_dados(self):
        try:
            # Chama todas as funções de consulta
            dados_beneficiarios = self.consultar_beneficiarios()
            dados_ies = self.consultar_ies()
            dados_cursos = self.consultar_cursos()
            dados_bolsas = self.consultar_bolsas()
            dados_regioes = self.consultar_regioes()
            dados_ufs = self.consultar_ufs()
            dados_municipios = self.consultar_municipios()
            
            # Formata os DataFrames
            df_beneficiarios = pd.DataFrame(dados_beneficiarios).rename(columns={'b': 'Beneficiario'})
            df_ies = pd.DataFrame(dados_ies).rename(columns={'ies': 'IES'})
            df_cursos = pd.DataFrame(dados_cursos).rename(columns={'c': 'Curso'})
            df_bolsas = pd.DataFrame(dados_bolsas).rename(columns={'bo': 'Bolsa'})
            df_regioes = pd.DataFrame(dados_regioes).rename(columns={'r': 'Regiao'})
            df_ufs = pd.DataFrame(dados_ufs).rename(columns={'uf': 'UF'})
            df_municipios = pd.DataFrame(dados_municipios).rename(columns={'m': 'Municipio'})
            
            # Retorna todos os DataFrames em um único DataFrame
            return pd.concat([df_beneficiarios, df_ies, df_cursos, df_bolsas, df_regioes, df_ufs, df_municipios], axis=1)
        
        except exceptions.TransientError as e:
            if "MemoryPoolOutOfMemoryError" in str(e):
                print("Erro: Limite de memória atingido. Tente novamente com uma carga de dados menor.")
            else:
                raise

    @staticmethod
    def formatar_dados(data_frame):
        # Percorre todas as colunas e descompacta os dicionários
        for col in data_frame.columns:
            data_frame = data_frame.join(data_frame[col].apply(pd.Series).add_prefix(col+'_'))

        # Remove as colunas originais
        data_frame = data_frame.drop(columns=data_frame.columns[:7])

        return data_frame


In [3]:
consulta = ConsultaDadosNeo4j()
dados_formatados = consulta.formatar_dados(consulta.consultar_todos_os_dados())

# Realize consultas

dados_formatados.head()

,Beneficiario_deficiente_fisico,Beneficiario_data_nascimento,Beneficiario_cpf,Beneficiario_raca,Beneficiario_sexo,Beneficiario_municipio_id,Beneficiario_0,IES_codigo_emec,IES_nome,IES_municipio_id,...,Bolsa_curso_id,Bolsa_ies_id,Bolsa_beneficiario_id,Regiao_nome_regiao,Regiao_0,UF_uf,UF_0,Municipio_nome_municipio,Municipio_uf_id,Municipio_0
0,BENEFICIARIO_DEFICIENTE_FISICO,DATA_NASCIMENTO,CPF_BENEFICIARIO,RACA_BENEFICIARIO,SEXO_BENEFICIARIO,NaN,NaN,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,NaN,...,NaN,NaN,NaN,Nome da Região,NaN,UF,NaN,Nome do Município,1.0,NaN
1,N,1998-04-27,107.XXX.XXX-31,Branca,F,8.0,NaN,10,PONTIFCIA UNIVERSIDADE CATLICA DO PARAN,8.0,...,12.0,11.0,10.0,SUL,NaN,PR,NaN,CURITIBA,7.0,NaN
2,N,2001-11-11,503.XXX.XXX-80,Parda,F,15.0,NaN,10,PONTIFCIA UNIVERSIDADE CATLICA DO PARAN,15.0,...,18.0,11.0,16.0,NORTE,NaN,RS,NaN,PORTO ALEGRE,14.0,NaN
3,N,1999-11-05,088.XXX.XXX-83,Parda,F,21.0,NaN,10,PONTIFCIA UNIVERSIDADE CATLICA DO PARAN,21.0,...,18.0,17.0,16.0,SUDESTE,NaN,SC,NaN,CAMPO MOURAO,7.0,NaN
4,N,1996-05-30,012.XXX.XXX-88,Preta,M,28.0,NaN,10,PONTIFCIA UNIVERSIDADE CATLICA DO PARAN,28.0,...,24.0,11.0,22.0,NORDESTE,NaN,PA,NaN,PARANACITY,7.0,NaN
